### Q-learning Agent for 2048

import and setup environment

In [1]:
import sys
sys.path.append('/path/to/gym-2048')

In [2]:
%cd c:\Users\Huhyr\iCloudDrive\Study\College\Senior Year\CSC370\Crack-2048\\gym-2048
%pip install -e .

c:\Users\Huhyr\iCloudDrive\Study\College\Senior Year\CSC370\Crack-2048\gym-2048
Obtaining file:///C:/Users/Huhyr/iCloudDrive/Study/College/Senior%20Year/CSC370/Crack-2048/gym-2048
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: gym-2048
    Found existing installation: gym-2048 0.0.5
    Uninstalling gym-2048-0.0.5:
      Successfully uninstalled gym-2048-0.0.5
  Running setup.py develop for gym-2048
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install gym
%pip install numpy
%pip install Pillow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import gym
import numpy as np
import gym_2048
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

env = gym.make('2048-v0')

env.reset()
env.render()


Score: 0
Highest: 2
[[0 0 0 0]
 [0 2 0 0]
 [0 0 2 0]
 [0 0 0 0]]


c:\Users\Huhyr\anaconda3\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:20: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: int32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
c:\Users\Huhyr\anaconda3\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:25: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(
c:\Users\Huhyr\anaconda3\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\Huhyr\anaconda3\envs\tf\lib

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


We start by creating a random strategy for the environment and test if the environment works. It will serves as a benchmark for our more advanced strategies to compare with.

In [4]:

# Function to play one game with random actions
def play_random_game(env):
    env.reset()
    done = False
    total_reward = 0

    while not done:
        
        # Choose a random action
        action = env.action_space.sample()

        # Perform the action and observe the results
        state, reward, done, _ = env.step(action)

        #render the game (optional)
        env.render()

        # Accumulate the reward
        total_reward += reward


    return total_reward

# Play a few games and calculate average score
num_games = 100
scores = [play_random_game(env) for _ in range(num_games)]
average_score = np.mean(scores)

print(f"Average score over {num_games} games: {average_score}")


c:\Users\Huhyr\anaconda3\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
c:\Users\Huhyr\anaconda3\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


Score: 0.0
Highest: 2
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 2 2 2]]
Score: 0.0
Highest: 2
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 2 2 2]]
Score: 0.0
Highest: 2
[[0 2 2 2]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Score: 0.0
Highest: 2
[[2 2 2 2]
 [0 0 0 0]
 [0 0 0 0]
 [2 0 0 0]]
Score: 8.0
Highest: 4
[[0 0 4 4]
 [0 0 0 2]
 [0 0 0 0]
 [0 0 0 2]]
Score: 12.0
Highest: 4
[[0 0 0 0]
 [0 0 0 0]
 [0 2 0 4]
 [0 0 4 4]]
Score: 20.0
Highest: 8
[[0 2 4 8]
 [0 0 0 0]
 [0 2 0 0]
 [0 0 0 0]]
Score: 24.0
Highest: 8
[[0 4 4 8]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Score: 32.0
Highest: 8
[[0 0 8 8]
 [0 0 2 2]
 [0 0 0 0]
 [0 0 0 0]]
Score: 32.0
Highest: 8
[[0 0 0 2]
 [0 0 0 0]
 [0 0 8 8]
 [0 0 2 2]]
Score: 32.0
Highest: 8
[[0 0 0 0]
 [0 2 0 2]
 [0 0 8 8]
 [0 0 2 2]]
Score: 56.0
Highest: 16
[[ 2  0  0  0]
 [ 0  0  0  4]
 [ 0  0  0 16]
 [ 0  0  0  4]]
Score: 56.0
Highest: 16
[[ 2  0  0  0]
 [ 4  2  0  0]
 [16  0  0  0]
 [ 4  0  0  0]]
Score: 56.0
Highest: 16
[[ 2  0  0  0]
 [ 4  2  0  0]
 [16  0  0  0]
 [ 4  0  0  0]]
Sc

: 

In [8]:

# Play a single game and log the moves
def play_random_game(env):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
        total_reward += reward

        print("State after action:")
        env.render()
        print(f"Reward: {reward}, Done: {done}")

        # Custom check for the end of the game can be implemented here

    return total_reward

score = play_random_game(env)
print(f"Final score: {score}")



State after action:
Score: 4.0
Highest: 4
[[0 0 0 4]
 [0 0 2 0]
 [0 0 0 0]
 [0 0 0 0]]
Reward: 4.0, Done: False
State after action:
Score: 4.0
Highest: 4
[[0 0 0 0]
 [0 0 0 0]
 [2 0 0 0]
 [0 0 2 4]]
Reward: 0.0, Done: False
State after action:
Score: 4.0
Highest: 4
[[0 0 0 0]
 [0 0 0 2]
 [0 0 0 0]
 [2 0 2 4]]
Reward: 0.0, Done: False
State after action:
Score: 8.0
Highest: 4
[[0 0 0 0]
 [0 0 0 2]
 [0 0 4 0]
 [0 0 4 4]]
Reward: 4.0, Done: False
State after action:
Score: 16.0
Highest: 8
[[0 0 0 4]
 [2 0 0 0]
 [4 0 0 0]
 [8 0 0 0]]
Reward: 8.0, Done: False
State after action:
Score: 16.0
Highest: 8
[[0 0 2 4]
 [0 0 0 2]
 [0 0 0 4]
 [0 0 0 8]]
Reward: 0.0, Done: False
State after action:
Score: 16.0
Highest: 8
[[0 0 2 4]
 [0 0 0 2]
 [0 0 0 4]
 [0 0 0 8]]
Reward: 0.0, Done: False
State after action:
Score: 16.0
Highest: 8
[[2 4 0 0]
 [2 0 0 0]
 [4 0 4 0]
 [8 0 0 0]]
Reward: 0.0, Done: False
State after action:
Score: 20.0
Highest: 8
[[4 4 4 0]
 [4 0 0 0]
 [8 0 0 0]
 [0 0 2 0]]
Reward: 4.0,

Now, we implement the Q learning agent

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0   # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.state_size, activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose =0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state, verbose = 0)[0])
            target_f = self.model.predict(state, verbose =0)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

def decode_state(encoded_state):
    """Decode the one-hot encoded state to the actual game state."""
    decoded = np.zeros((4, 4), dtype=int)
    for i in range(4):
        for j in range(4):
            one_hot = encoded_state[i, j]
            if np.sum(one_hot) > 0:  # If there is a tile present
                tile_value = 2 ** (np.argmax(one_hot) + 1)
                decoded[i, j] = tile_value
    return decoded.flatten()

env = gym.make('2048-v0')
state_size = 16  # Flattened decoded state size
batch_size = 32
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

REPLAY_EVERY = 32  # Perform a replay every 50 actual game steps
MAX_EPISODE_LENGTH = 5000  # Maximum steps per episode
game_step_count = 0

for e in range(300):  # number of episodes
    encoded_state = env.reset()
    state = decode_state(encoded_state)
    state = np.reshape(state, [1, state_size])  # Flatten the decoded state

    for time in range(MAX_EPISODE_LENGTH):  # Limited episode length
        action = agent.act(state)
        encoded_next_state, reward, done, _ = env.step(action)
        next_state = decode_state(encoded_next_state)
        next_state = np.reshape(next_state, [1, state_size])

        reward = reward if not done else -10
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        game_step_count += 1

        if done or time == MAX_EPISODE_LENGTH - 1:
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, 1000, time, agent.epsilon))
            env.render()
            break

        if len(agent.memory) > batch_size and game_step_count % REPLAY_EVERY == 0:
            agent.replay(batch_size)
            game_step_count = 0

# Optionally save the model
agent.save("2048-dqn.h5")


episode: 0/1000, score: 256, e: 0.97
Score: 2364.0
Highest: 256
[[  2  32   4   2]
 [  4   2  16   8]
 [ 16  64 256   2]
 [  4   2   8  16]]
episode: 1/1000, score: 134, e: 0.95
Score: 888.0
Highest: 64
[[ 2  4 16  4]
 [ 4  8 64  8]
 [ 8 16  8 32]
 [ 2 64  4  2]]
episode: 2/1000, score: 102, e: 0.93
Score: 608.0
Highest: 64
[[ 2  4  8  4]
 [ 8 64 32 16]
 [16  2  8  4]
 [ 2  4 16  2]]
episode: 3/1000, score: 268, e: 0.9
Score: 2764.0
Highest: 256
[[  2   4   8   2]
 [  4  32 128   4]
 [  8  16 256  16]
 [  2   4   8   4]]
episode: 4/1000, score: 159, e: 0.87
Score: 1296.0
Highest: 128
[[  4  32   8   2]
 [  2 128  16   4]
 [  8  64   4   8]
 [  2   4   8   4]]
episode: 5/1000, score: 154, e: 0.85
Score: 1100.0
Highest: 128
[[  2  16   4   2]
 [ 16 128  16   8]
 [  8   4  32   4]
 [  4  16   4   2]]
episode: 6/1000, score: 76, e: 0.84
Score: 432.0
Highest: 32
[[16  4  2  4]
 [ 8  2 16  8]
 [ 2 16 32  4]
 [ 4 32  8  2]]
episode: 7/1000, score: 109, e: 0.83
Score: 692.0
Highest: 64
[[ 2 64

: 

Training

In [4]:
# Training Parameters
total_training_episodes = 10
batch_size = 32

# Initialize gym environment and the agent
env = gym.make('2048-v0')
state_shape = env.observation_space.shape
flattened_state_size = np.prod(state_shape)  # Calculate the flattened state size
action_size = env.action_space.n
agent = DQNAgent(flattened_state_size, action_size)

for episode in range(total_training_episodes):
    state = env.reset()
    state = np.reshape(state, [1, flattened_state_size])

    for time_step in range(500):  # Define a maximum number of steps per episode
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, flattened_state_size])
        
        agent.remember(state, action, reward, next_state, done)

        state = next_state
        if done:
            break

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    
    print(f"Episode: {episode + 1}/{total_training_episodes}, Score: {time_step}")



c:\Users\Huhyr\anaconda3\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
c:\Users\Huhyr\anaconda3\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


KeyboardInterrupt: 

Testing

In [1]:


# Load the pre-trained model if available
# agent.load("path_to_your_model.h5")

# Testing Loop
scores = []  # List to keep track of scores for each game
for game in range(100):
    state = env.reset()
    state = decode_state(state)
    state = np.reshape(state, [1, state_size])
    score = 0

    while True:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = decode_state(next_state)
        next_state = np.reshape(next_state, [1, state_size])

        score += reward
        state = next_state

        if done:
            break

    scores.append(score)
    print(f"Game {game+1}: Score = {score}")

# Analysis of results
average_score = np.mean(scores)
print(f"Average Score over 100 games: {average_score}")


NameError: name 'env' is not defined

: 